In [ ]:
pip install requests beautifulsoup4

In [34]:
#Importing Libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Defining my function for extracting auction data from html elements using beautifulsoup
# The div classes containing the information I needed were nested in seperate classes.
# Because of this I chose to create multiple loops that extract and append the nested information into seperate lists.
def extract_data(soup):
    info = []
    for s in soup.find_all(class_="Box-sc-15se88d-0 GridColumns__Cell-sc-1g9p6xx-1 fRyjXC hVgfsm"):
        divs = s.find_all("div")
        while len(divs) < 3:
            divs.append(None)
        row = {
            'Title': divs[0].text.strip() if divs[0] else None,
            'Medium': divs[1].text.strip() if divs[1] else None,
            'Dimensions': divs[2].text.strip() if divs[2] else None
        }
        info.append(row)

    info2 = []
    for s in soup.find_all(class_="Box-sc-15se88d-0 GridColumns__Cell-sc-1g9p6xx-1 fRyjXC dtgSid"):
        divs = s.find_all("div")
        while len(divs) < 2:
            divs.append(None)
        row = {
            'Date': divs[0].text.strip() if divs[0] else None,
            'Auction House': divs[1].text.strip() if divs[1] else None
        }
        info2.append(row)

    return info, info2
# This is my functionfor tackling the pagination issue.
# I started by finding the base url and then comparing it to the second and third pages.
# I then increwmented the pages and looped it to feed it back into the html parser
# for testing I added a maximum page limiter

def scrape_data(max_pages=5):  # Set page limit to 5 for testing
    base_url = "https://www.artsy.net/artist/roy-lichtenstein/auction-results"
    page_number = 1
    all_results = []
    all_results2 = []

    while page_number <= max_pages:
        url = f"{base_url}?hide_upcoming=false&page={page_number}&allow_empty_created_dates=true&metric=in&currency=&include_estimate_range=false&include_unknown_prices=true&allow_unspecified_sale_dates=true"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        if soup.find(text="No results found"):
            break

        data, data2 = extract_data(soup)
        all_results.extend(data)
        all_results2.extend(data2)
        page_number += 1

    return all_results, all_results2

# Calling the scraping function with a limit on the number of pages and create DataFrames
data_list, data_list2 = scrape_data(5)  # limit to 5 pages for testing
df = pd.DataFrame(data_list)
df2 = pd.DataFrame(data_list2)

# Merging and the dataframes into one
df3 = pd.concat([df, df2], axis=1)
df3



<ipython-input-34-e55cddad0c67>:46: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  if soup.find(text="No results found"):


,Title,Medium,Dimensions,Date,Auction House
0,"Sunrise, 1965",lithograph in colours on paper,44.0 x 59.0 cm,"Apr 23, 2024",Christie's
1,Temple (C. II.3),"Offset lithograph in colors, on cream wove pap...",I. 23 1/8 x 17 1/8 in. (58.7 x 43.5 cm)\r\nS. ...,"Apr 18, 2024",Phillips • New York
2,Real Estate (Inscribed letterhead) (see C. 88),"Letterhead stationery printed in black, on wov...",9 7/8 x 8 in. (25.1 x 20.3 cm),"Apr 18, 2024",Phillips • New York
3,"Roommates, from Nudes Series (C. 282)","Monumental relief print in colors, on Rives BF...",I. 57 1/4 x 45 in. (145.4 x 114.3 cm)\r\nS. 64...,"Apr 18, 2024",Phillips • New York
4,"Seascape, from Landscapes Series (G. 1252, C. ...","Lithograph, woodcut and screenprint in colors,...",I. 37 3/8 x 52 1/2 in. (94.9 x 133.4 cm)\r\nS....,"Apr 18, 2024",Phillips • New York
...,...,...,...,...,...
245,"Entablature IV, 1976",screenprint in colors with collage and embossi...,51.2 x 96.6 cm,"Apr 12, 2023",Sotheby's
246,"Entablature X, 1976",screenprint and lithograph printed in colors w...,53.4 x 96.4 cm,"Apr 12, 2023",Sotheby's
247,"American Indian Theme IV, 1980",woodcut and lithograph printed in colors on ha...,72.1 x 71.0 cm,"Apr 12, 2023",Sotheby's
248,"Two Figures with Teepee, 1980","etching, aquatint and engraving printed in col...",24.7 x 22.1 cm,"Apr 12, 2023",Sotheby's


In [23]:
pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 30.6 MB/s eta 0:00:00


In [24]:
#importing libraries

import mysql.connector
from mysql.connector import Error

In [33]:
import mysql.connector

# Establish connection to my database in Dbeaver
connection = mysql.connector.connect(
    host='isba-dev-01.crw88eg88uar.us-east-1.rds.amazonaws.com',
    user='admin',
    password='isba_4715',
    database='artworks'
)

# Creating a cursor to run queries
cursor = connection.cursor()

# this code was from my API ETL using the same database.
# I recognize in the prompt that the database should be named sql_project, I felt it was redundant to change it since I Would have to reupload all by data
cursor.execute("CREATE DATABASE IF NOT EXISTS artworks")
cursor.execute("USE artworks")

# Here I am createing a table to store the auction information
cursor.execute("""
    CREATE TABLE IF NOT EXISTS roy_lichtenstein_auctions (
        Title VARCHAR(255),
        Medium VARCHAR(255),
        Dimensions VARCHAR(255),
        Date VARCHAR(255),
        Auction_House VARCHAR(255)
    )
""")

# Here I am preparing the INSERT INTO statement which is allows me to insert my data into the approriate columns
insert_query = """
    INSERT INTO roy_lichtenstein_auctions (Title, Medium, Dimensions, Date, Auction_House)
    VALUES (%s, %s, %s, %s, %s)
"""

# This is a loop that iterates through my webscrabed dataframe inserting row by row into the the table
for index, row in df3.iterrows():
    cursor.execute(insert_query, (row['Title'], row['Medium'], row['Dimensions'], row['Date'], row['Auction House']))

# Commits the changes to the database
connection.commit()

# Closes the cursor and connection
cursor.close()
connection.close()

print("Artwork information successfully inserted into the MySQL database.")


Artwork information successfully inserted into the MySQL database.
